## LSTMで自然言語処理
- https://www.atmarkit.co.jp/ait/articles/1801/30/news139.html
- 自然言語をディープラーニングで扱う場合、何らかの方法で単語をベクトルデータに変換する必要がある。しゃれた言い方では、n次元実数空間への埋め込みであるが、要はn個の実数の組に対応させることである。とにかく、数字に変換しないと始まらない。
- one-hotベクトルへの変換...自然言語のベクトル化手法のうち最もかんたんな方法。語彙数がNであるとき、各単語に0からN-1までのインデックスiを振る。各単語に対し、i次元の値が1で、それ以外の値が0のベクトル（1つの次元だけが1で他が0のため、one-hotベクトルと呼ばれる）に対応させると、確かにN次元実数空間への埋め込みが実現できる。
- kerasには「Embedding」レイヤーというものが標準で用意されており、指定された次元への埋め込みベクトルを生成してくれる。
- word2vec...単語間の関連性を対応するベクトル間演算（足し引き）で表現できるようにしようというもの、word to vector、自然言語のベクトル化手法のうち語彙数より少ない次元Embedding数のベクトル化手法。CBOW（Continuous Bag-of-Words Model）とContinuous Skip-gram Model（以下、skip-gramと表記）の2種類の手法がある。
 - CBOW...前後の単語からターゲットの単語を予測しようというもの
 - skip-gram...特定の単語が与えられたときに、その前後の単語を当てようというもの

### import宣言

In [4]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
import random
import numpy.random as nr
import sys
import h5py
import keras
import math


from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Masking
from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.layers.recurrent import SimpleRNN
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_uniform
from keras.initializers import uniform
from keras.initializers import orthogonal
from keras.initializers import TruncatedNormal
from keras.optimizers import RMSprop
from keras import regularizers
from keras.constraints import maxnorm, non_neg
from keras.utils.data_utils import get_file
from keras.utils import np_utils

Using TensorFlow backend.


In [5]:
# 元データ
df1 = csv.reader(open('rampo_separate.csv', 'r'))
df2 = csv.reader(open('rampo_separate2.csv', 'r'))
df3 = csv.reader(open('rampo_separate3.csv', 'r'))


data1 = [ v for v in df1]
data2 = [ v for v in df2]
data3 = [ v for v in df3]

mat1 = np.array(data1)
mat2 = np.array(data2)
mat3 = np.array(data3)

mat = np.r_[mat1[:,0], mat2[:,0], mat3[:,0]]
print(mat.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'rampo_separate.csv'

In [ ]:


words = sorted(list(set(mat)))
cnt = np.zeros(len(words))

print('total words:', len(words))
word_indices = dict((w, i) for i, w in enumerate(words))  # 単語をキーにインデックス検索
indices_word = dict((i, w) for i, w in enumerate(words))  # インデックスをキーに単語を検索

# 単語の出現数をカウント
for j in range (0, len(mat)):
    cnt[word_indices[mat[j]]] += 1

# 出現頻度の少ない単語を「UNK」で置き換え
words_unk = []                           # 未知語一覧

for k in range(0, len(words)):
    if cnt[k] <= 3 :
        words_unk.append(words[k])
        words[k] = 'UNK'

print('低頻度語数:', len(words_unk))    # words_unkはunkに変換された単語のリスト

words = sorted(list(set(words)))
print('total words:', len(words))
word_indices = dict((w, i) for i, w in enumerate(words))  # 単語をキーにインデックス検索
indices_word = dict((i, w) for i, w in enumerate(words))  # インデックスをキーに単語を検索

In [ ]:


maxlen = 10                   # 前後の語数

mat_urtext = np.zeros((len(mat), 1), dtype=int)
for i in range(0, len(mat)):
    if mat[i] in word_indices :        
        mat_urtext[i, 0] = word_indices[mat[i]]
    else:
        mat_urtext[i, 0] = word_indices['UNK']

print(mat_urtext.shape)

len_seq = len(mat_urtext) - maxlen
data = []
target = []
for i in range(maxlen, len_seq):
    data.append(mat_urtext[i])
    target.extend(mat_urtext[i-maxlen:i])
    target.extend(mat_urtext[i+1:i+1+maxlen])

x_train = np.array(data).reshape(len(data), 1)
t_train = np.array(target).reshape(len(data), maxlen*2)

z = zip(x_train, t_train)
nr.seed(12345)
nr.shuffle(z)                 # シャッフル
x_train, t_train = zip(*z)

x_train = np.array(x_train).reshape(len(data), 1)
t_train = np.array(t_train).reshape(len(data), maxlen*2)

print(x_train.shape, t_train.shape)

In [ ]:
# ニューラルネットワーク本体


class Prediction :
    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim
    def create_model(self):
        model = Sequential()
        model.add(Embedding(self.input_dim, self.output_dim, input_length=1, embeddings_initializer=uniform(seed=20170719)))
        model.add(Flatten())
        model.add(Dense(self.input_dim, use_bias=False, kernel_initializer=glorot_uniform(seed=20170719)))
        model.add(Activation("softmax"))
        model.compile(loss="categorical_crossentropy", optimizer="RMSprop", metrics=['categorical_accuracy'])
        print('#2')
        return model
   
  # 学習
    def train(self, x_train, t_train,batch_size, epochs, maxlen, emb_param) :
        early_stopping = EarlyStopping(monitor='categorical_accuracy', patience=1, verbose=1)
        print ('#1', t_train.shape)
        model = self.create_model()
        #model.load_weights(emb_param)    # 埋め込みパラメーターセット。ファイルをロードして学習を再開したいときに有効にする
        print ('#3')
        model.fit(x_train, t_train, batch_size=batch_size, epochs=epochs, verbose=1,
              shuffle=True, callbacks=[early_stopping], validation_split=0.0)
        return model

In [ ]:
#メイン処理


vec_dim = 100
epochs = 10
batch_size = 200
input_dim = len(words)
output_dim = vec_dim

emb_param = 'param_skip_gram_2_1.hdf5'    # 学習済みパラメーターファイル名
prediction = Prediction(input_dim, output_dim)
row = t_train.shape[0]

t_one_hot = np.zeros((row, input_dim), dtype='int8')    # ラベルデータをN-hot化
for i in range(0, row) :
    for j in range(0, maxlen*2):
        t_one_hot[i, t_train[i,j]] = 1

x_train = x_train.reshape(row,1)
model = prediction.train(x_train, t_one_hot, batch_size, epochs, maxlen, emb_param)

model.save_weights(emb_param)           # 学習済みパラメーターセーブ

In [ ]:
# 埋め込みベクトルの評価


param_lstm = model.get_weights()
param = param_lstm[0]
word0 = '一'
word1 = '１'
word2 = '２'
vec0 = param[word_indices[word0],:]
vec1 = param[word_indices[word1],:]
vec2 = param[word_indices[word2],:]

vec = vec0 - vec1 + vec2
vec_norm = math.sqrt(np.dot(vec, vec))

w_list = [word_indices[word0], word_indices[word1], word_indices[word2]]
dist = -1.0
m = 0
for j in range(0, 5) :
    dist = -1.0
    m = 0
    for i in range(0, len(words)) :
        if i not in w_list :
            dist0 = np.dot(vec, param[i,:])
            dist0 = dist0 / vec_norm / math.sqrt(np.dot(param[i,:], param[i,:]))
            if dist < dist0 :
                dist = dist0
                m = i
    print('第' + str(j+1) + '候補:')
    print('コサイン類似度=', dist, ' ', m, ' ', indices_word[m])
    w_list.append(m)